In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from polimi_scraper.config import DataPath

## Individual data sources

### Classroom

In [3]:
classroom_df = pd.read_parquet(DataPath.RAW_CLASSROOMS).set_index("codice_patrimonio")
classroom_df

,name,address,plugs
codice_patrimonio,,,
CRG0102001001,A.1.1-CR,"Via Sesto, 39 - 26100 - Cremona (CR)",False
CRG0102002001,A.2.1-CR,"Via Sesto, 39 - 26100 - Cremona (CR)",False
CRG0102002002,A.2.2-CR,"Via Sesto, 39 - 26100 - Cremona (CR)",False
CRG0101000038b,AULA STUDIO GIALLA 1,"Via Sesto, 41 - 26100 - Cremona (CR)",False
CRG0101000038a,AULA STUDIO GIALLA 2,"Via Sesto, 41 - 26100 - Cremona (CR)",False
...,...,...,...
MNI0101001025,A 1.2,"Via Scarsellini, 15 - 46100 - Mantova (MN)",False
MNI0101001001,A.1.3,"Via Scarsellini, 15 - 46100 - Mantova (MN)",False
MNI0101001073,A.1.4,"Via Scarsellini, 15 - 46100 - Mantova (MN)",False


### Building

In [4]:
def estimate_building_center(paths: list[list[dict]]) -> tuple[float, float]:
    x, y = [], []
    for point in paths[0]:
        x.append(point["lng"])
        y.append(point["lat"])
    return pd.Series([(min(x) + max(x)) / 2, (min(y) + max(y)) / 2], index=["x", "y"])

In [5]:
building_df = pd.read_parquet(DataPath.RAW_PLACES_DIR / "edificio.parquet").set_index("codice_patrimonio")
building_df["paths"].iloc[0][0]

array([{'lat': 45.5030062148, 'lng': 9.1561159215},
       {'lat': 45.5029047009, 'lng': 9.1549478194},
       {'lat': 45.5028407846, 'lng': 9.1549598894},
       {'lat': 45.5028445444, 'lng': 9.1550121925},
       {'lat': 45.5027524297, 'lng': 9.155030968},
       {'lat': 45.5027486699, 'lng': 9.1549786649},
       {'lat': 45.5026922731, 'lng': 9.1549907348},
       {'lat': 45.5027947273, 'lng': 9.1561588368}], dtype=object)

In [6]:
building_coordinates_df = building_df["paths"].apply(estimate_building_center)
building_coordinates_df

,x,y
codice_patrimonio,,
MIB0124,9.155553,45.502849
MIA0110,9.229917,45.479135
MIA0105,9.228652,45.477452
MIA0203,9.228173,45.480008
MIA0302,9.231182,45.478877
...,...,...
MIA0115,9.228056,45.477224
MIC0401,9.219920,45.438444
MIB0119,9.154349,45.502826


### Room polygons

In [18]:
polygons_df = pd.read_parquet(DataPath.RAW_POLYGONS)
polygons_df

,codice_patrimonio,x,y
0,COE040100S028,38.62870,11.99580
1,COE040100S029,38.62870,10.85415
2,COE040100S019,37.28940,19.47040
3,COE040100S006,6.67450,22.98930
4,COE040100S007,9.38015,27.24425
...,...,...,...
25557,PCL010100V015,50.90710,18.59805
25558,PCL010100V012,37.27550,15.70600
25559,PCL010100V013,48.97860,18.84365
25560,PCL010100V014,49.04015,17.67600


A unique numerical identifier for each floor is required for the linear programming model, since we wish to determine when two rooms belong to different floors by subtracting their floor identifiers

In [20]:
floor_series = polygons_df["codice_patrimonio"].str[7:10].sort_values()
floor_series.unique()

array(['000', '001', '002', '003', '004', '005', '006', '007', '008',
       '009', '00R', '00S', '00V', '00Z', '010', '011', '01I', '02I',
       '03I', 'S00', 'S01', 'S03', 'S0R', 'S0S'], dtype=object)

In [21]:
floor_encoding = {floor: i for i, floor in enumerate(floor_series.sort_values().unique())}
floor_encoding

{'000': 0,
 '001': 1,
 '002': 2,
 '003': 3,
 '004': 4,
 '005': 5,
 '006': 6,
 '007': 7,
 '008': 8,
 '009': 9,
 '00R': 10,
 '00S': 11,
 '00V': 12,
 '00Z': 13,
 '010': 14,
 '011': 15,
 '01I': 16,
 '02I': 17,
 '03I': 18,
 'S00': 19,
 'S01': 20,
 'S03': 21,
 'S0R': 22,
 'S0S': 23}

In [22]:
polygons_df["floor"] = polygons_df["codice_patrimonio"].str[7:10].map(floor_encoding)
polygons_df

,codice_patrimonio,x,y,floor
0,COE040100S028,38.62870,11.99580,11
1,COE040100S029,38.62870,10.85415,11
2,COE040100S019,37.28940,19.47040,11
3,COE040100S006,6.67450,22.98930,11
4,COE040100S007,9.38015,27.24425,11
...,...,...,...,...
25557,PCL010100V015,50.90710,18.59805,12
25558,PCL010100V012,37.27550,15.70600,12
25559,PCL010100V013,48.97860,18.84365,12
25560,PCL010100V014,49.04015,17.67600,12


We derive some external keys from `codice_patrimonio` to merge with other data sources

In [11]:
polygons_df["sede"] = polygons_df["codice_patrimonio"].str[:3]
polygons_df["building"] = polygons_df["codice_patrimonio"].str[:7]
polygons_df = polygons_df.rename(columns={"codice_patrimonio": "room"})
polygons_df

,room,x,y,floor,sede,building
0,COE040100S028,38.62870,11.99580,11,COE,COE0401
1,COE040100S029,38.62870,10.85415,11,COE,COE0401
2,COE040100S019,37.28940,19.47040,11,COE,COE0401
3,COE040100S006,6.67450,22.98930,11,COE,COE0401
4,COE040100S007,9.38015,27.24425,11,COE,COE0401
...,...,...,...,...,...,...
25557,PCL010100V015,50.90710,18.59805,12,PCL,PCL0101
25558,PCL010100V012,37.27550,15.70600,12,PCL,PCL0101
25559,PCL010100V013,48.97860,18.84365,12,PCL,PCL0101
25560,PCL010100V014,49.04015,17.67600,12,PCL,PCL0101


## Merge

In [12]:
processed_classrooms_df = (
    polygons_df.join(
        building_coordinates_df,
        on="building",
        lsuffix="_room",
        rsuffix="_building",
    )
    .join(classroom_df, on="room", how="inner")
    .drop(columns=["room", "building"])
    .set_index(["sede", "name"])
)
processed_classrooms_df

x_room    y_room  floor  x_building  \
sede name                                                                
CRG  A.1.1-CR                    24.08550  31.19980      1   10.002273   
     A.2.1-CR                    20.22040  31.92690      2   10.002273   
     A.2.2-CR                    30.95040  32.18390      2   10.002273   
     AULA STUDIO GIALLA 1        62.48000  54.67070      0   10.001991   
     AULA STUDIO GIALLA 2        63.25000  42.04570      0   10.001991   
...                                   ...       ...    ...         ...   
PCL  SALA RIUNIONI GINO MARCHET  57.06485  20.80980      1    9.701878   
     AULA M                      41.25155  11.57720     12    9.701878   
     AULA S                      54.64770  12.53455     12    9.701878   
     AULA P                      27.41125   8.05945     12    9.701878   
     AULA STUDIO                 53.61580  30.24440     12    9.701878   

                                 y_building  \
sede name                                     
CRG  A.1.1-CR                     45.146151   
     A.2.1-CR                     45.146151   
     A.2.2-CR                     45.146151   
     AULA STUDIO GIALLA 1         45.146616   
     AULA STUDIO GIALLA 2         45.146616   
...                                     ...   
PCL  SALA RIUNIONI GINO MARCHET   45.048264   
     AULA M                       45.048264   
     AULA S                       45.048264   
     AULA P                       45.048264   
     AULA STUDIO                  45.048264   

                                                                    address  \
sede name                                                                     
CRG  A.1.1-CR                          Via Sesto, 39 - 26100 - Cremona (CR)   
     A.2.1-CR                          Via Sesto, 39 - 26100 - Cremona (CR)   
     A.2.2-CR                          Via Sesto, 39 - 26100 - Cremona (CR)   
     AULA STUDIO GIALLA 1              Via Sesto, 41 - 26100 - Cremona (CR)   
     AULA STUDIO GIALLA 2              Via Sesto, 41 - 26100 - Cremona (CR)   
...                                                                     ...   
PCL  SALA RIUNIONI GINO MARCHET  Via Scalabrini, 76 - 29100 - Piacenza (PC)   
     AULA M                      Via Scalabrini, 76 - 29100 - Piacenza (PC)   
     AULA S                      Via Scalabrini, 76 - 29100 - Piacenza (PC)   
     AULA P                      Via Scalabrini, 76 - 29100 - Piacenza (PC)   
     AULA STUDIO                 Via Scalabrini, 76 - 29100 - Piacenza (PC)   

                                 plugs  
sede name                               
CRG  A.1.1-CR                    False  
     A.2.1-CR                    False  
     A.2.2-CR                    False  
     AULA STUDIO GIALLA 1        False  
     AULA STUDIO GIALLA 2        False  
...                                ...  
PCL  SALA RIUNIONI GINO MARCHET  False  
     AULA M                      False  
     AULA S                      False  
     AULA P                      False  
     AULA STUDIO                 False  

[353 rows x 7 columns]

In [13]:
%%timeit
processed_classrooms_df.loc[("MIA", "B.2.1")]

29.7 μs ± 249 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
%%timeit
processed_classrooms_df.loc["MIA"].loc["B.2.1"]

184 μs ± 1.68 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Querying over the multi-index is significantly faster than querying over each index individually

In [15]:
processed_classrooms_df.describe()

,x_room,y_room,floor,x_building,y_building
count,353.000000,353.000000,353.000000,353.000000,353.000000
mean,40.361201,35.390399,2.277620,9.347884,45.451653
std,25.240259,24.739687,3.718349,0.329480,0.190799
min,4.087500,4.302500,0.000000,9.154317,45.046377
25%,17.337500,17.425000,0.000000,9.166840,45.477446
50%,35.951350,31.199800,1.000000,9.228173,45.479848
75%,57.064850,52.147600,2.000000,9.396380,45.504705
max,124.236550,297.170600,19.000000,10.789112,45.850246


In [16]:
processed_classrooms_df.to_parquet(DataPath.CLASSROOMS)